In [15]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\fidel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fidel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fidel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\fidel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [16]:
data = fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame({
    'text': data.data,
    'target': data.target,
    'target_name': [data.target_names[i] for i in data.target]
})
df.head()


,text,target,target_name
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware


# Check if target_names occur in the texts

In [17]:
# Function to count how many documents contain the target_name at least once
def count_target_mentions(dataframe):
    count = 0
    for _, row in dataframe.iterrows():
        name = row['target_name'].lower()
        text = row['text'].lower()
        if name in text:
            count += 1
    return count

total_mentions = count_target_mentions(df)
print(f"\nDocuments where the target name appears in the text: {total_mentions} out of {len(df)}")



Documents where the target name appears in the text: 350 out of 18846


# Remove target_names

In [18]:
def remove_target_name(row):
    name = re.escape(row['target_name'])  # Handle special chars
    pattern = re.compile(name, re.IGNORECASE)
    return pattern.sub('', row['text'])

# Overwrite the 'text' column directly
df['text'] = df.apply(remove_target_name, axis=1)

total_mentions = count_target_mentions(df)
print(f"\nDocuments where the target name appears in the text: {total_mentions} out of {len(df)}")

#print(df.head())



Documents where the target name appears in the text: 0 out of 18846


# Lemmatization (nltk & pywsd

In [19]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # tokenize & lowercase
    filtered = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return " ".join(filtered)

In [20]:
df['text'] = df['text'].apply(preprocess_text)

In [22]:
df.head()
#df.to("20newsgroup_preprocessed.pkl")
df.to_csv("20newsgroup_preprocessed.csv", index=False)